# Amazon Redshift Data Warehouse Workshop

This hands-on workshop guides you through creating, configuring, and using an Amazon Redshift cluster for data warehousing. Each cell contains clear instructions that can be executed sequentially.

## Prerequisites

- AWS account with appropriate permissions
- AWS CLI configured with access credentials
- Python 3.8+ with the following packages installed:
  - boto3
  - pandas
  - psycopg2 or psycopg2-binary

## 1. Environment Setup

### 1.1 Import Required Libraries

In [1]:
import boto3
import pandas as pd
import time
import json
from botocore.exceptions import ClientError
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import os

/Users/sanjij/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


### 1.2 Set Configuration Parameters

In [2]:
# AWS Configuration
region = 'ap-south-1'  # Change to your preferred region

# Redshift Cluster Configuration
cluster_identifier = 'redshift-workshop'
node_type = 'dc2.large'  # For workshop, using smaller node type
number_of_nodes = 2
db_name = 'dwh'
master_username = 'admin'
master_password = 'Redshift123!'  # Use a secure password in production

# VPC Configuration - will use default VPC for simplicity
vpc_id = 'default'  # Will be populated later

# S3 Configuration
s3_bucket = f'redshift-workshop-{int(time.time())}'  # Unique bucket name
sample_data_prefix = 'sample-data/'

## 2. Create Resources

### 2.1 Create IAM Role for Redshift

In [3]:
def create_redshift_service_role():
    """Create IAM role for Redshift to access S3"""
    iam = boto3.client('iam', region_name=region)
    
    # Define trust policy
    trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {"Service": "redshift.amazonaws.com"},
                "Action": "sts:AssumeRole"
            }
        ]
    }
    
    try:
        # Create the role
        response = iam.create_role(
            RoleName='RedshiftWorkshopRole',
            AssumeRolePolicyDocument=json.dumps(trust_policy),
            Description='Role for Redshift workshop to access S3'
        )
        
        role_arn = response['Role']['Arn']
        print(f"Created IAM role: RedshiftWorkshopRole with ARN: {role_arn}")
        
        # Attach S3 read policy
        iam.attach_role_policy(
            RoleName='RedshiftWorkshopRole',
            PolicyArn='arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'
        )
        
        print("Attached S3 read policy to role")
        return role_arn
    
    except ClientError as e:
        if e.response['Error']['Code'] == 'EntityAlreadyExists':
            print("IAM role RedshiftWorkshopRole already exists")
            response = iam.get_role(RoleName='RedshiftWorkshopRole')
            return response['Role']['Arn']
        else:
            print(f"Error creating IAM role: {e}")
            raise

# Create the IAM role
redshift_role_arn = create_redshift_service_role()

Created IAM role: RedshiftWorkshopRole with ARN: arn:aws:iam::359373501475:role/RedshiftWorkshopRole
Attached S3 read policy to role


### 2.2 Create S3 Bucket and Upload Sample Data

In [4]:
def create_s3_bucket_and_sample_data():
    """Create S3 bucket and generate sample data"""
    s3 = boto3.client('s3', region_name=region)
    
    # Create bucket
    try:
        if region == 'us-east-1':
            s3.create_bucket(Bucket=s3_bucket)
        else:
            s3.create_bucket(
                Bucket=s3_bucket,
                CreateBucketConfiguration={'LocationConstraint': region}
            )
        print(f"Created S3 bucket: {s3_bucket}")
    except ClientError as e:
        print(f"Error creating bucket: {e}")
        raise
    
    # Generate sample customers data
    customers_data = pd.DataFrame({
        'customer_id': range(1, 101),
        'customer_name': [f'Customer {i}' for i in range(1, 101)],
        'email': [f'customer{i}@example.com' for i in range(1, 101)],
        'city': ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix'] * 20,
        'state': ['NY', 'CA', 'IL', 'TX', 'AZ'] * 20,
        'country': ['USA'] * 100,
        'registration_date': pd.date_range(start='2020-01-01', periods=100, freq='D')
    })
    
    # Generate sample products data
    products_data = pd.DataFrame({
        'product_id': range(1, 51),
        'product_name': [f'Product {i}' for i in range(1, 51)],
        'category': ['Electronics', 'Clothing', 'Home', 'Books', 'Toys'] * 10,
        'price': [round(i * 9.99, 2) for i in range(1, 51)],
        'cost': [round(i * 5.99, 2) for i in range(1, 51)]
    })
    
    # Generate sample sales data
    import numpy as np
    np.random.seed(42)
    
    sales_data = []
    for i in range(1, 1001):
        customer_id = np.random.randint(1, 101)
        product_id = np.random.randint(1, 51)
        product_price = products_data.loc[product_id-1, 'price']
        quantity = np.random.randint(1, 5)
        
        sales_data.append({
            'sale_id': i,
            'customer_id': customer_id,
            'product_id': product_id,
            'sale_date': pd.Timestamp('2023-01-01') + pd.Timedelta(days=np.random.randint(0, 365)),
            'quantity': quantity,
            'unit_price': product_price,
            'total_amount': quantity * product_price
        })
    
    sales_df = pd.DataFrame(sales_data)
    
    # Save to local files first
    customers_data.to_csv('customers.csv', index=False)
    products_data.to_csv('products.csv', index=False)
    sales_df.to_csv('sales.csv', index=False)
    
    # Upload to S3
    s3_resource = boto3.resource('s3')
    s3_resource.Object(s3_bucket, f"{sample_data_prefix}customers.csv").upload_file('customers.csv')
    s3_resource.Object(s3_bucket, f"{sample_data_prefix}products.csv").upload_file('products.csv')
    s3_resource.Object(s3_bucket, f"{sample_data_prefix}sales.csv").upload_file('sales.csv')
    
    print(f"Uploaded sample data to s3://{s3_bucket}/{sample_data_prefix}")
    
    # Clean up local files
    os.remove('customers.csv')
    os.remove('products.csv')
    os.remove('sales.csv')
    
    return True

# Create bucket and upload sample data
create_s3_bucket_and_sample_data()

Created S3 bucket: redshift-workshop-1743684928
Uploaded sample data to s3://redshift-workshop-1743684928/sample-data/


True

### 2.3 Create Redshift Cluster

In [5]:
def create_redshift_cluster():
    """Create a Redshift cluster"""
    redshift = boto3.client('redshift', region_name=region)
    
    # Get default VPC ID
    ec2 = boto3.client('ec2', region_name=region)
    vpcs = ec2.describe_vpcs(
        Filters=[{'Name': 'isDefault', 'Values': ['true']}]
    )
    
    if not vpcs['Vpcs']:
        raise Exception("No default VPC found in this account/region")
    
    global vpc_id
    vpc_id = vpcs['Vpcs'][0]['VpcId']
    print(f"Using default VPC: {vpc_id}")
    
    # Create security group for Redshift
    try:
        sg_response = ec2.create_security_group(
            GroupName='redshift-workshop-sg',
            Description='Security group for Redshift workshop',
            VpcId=vpc_id
        )
        security_group_id = sg_response['GroupId']
        
        # Add ingress rule to allow connections
        ec2.authorize_security_group_ingress(
            GroupId=security_group_id,
            IpPermissions=[
                {
                    'IpProtocol': 'tcp',
                    'FromPort': 5439,
                    'ToPort': 5439,
                    'IpRanges': [{'CidrIp': '0.0.0.0/0'}]  # In production, restrict this!
                }
            ]
        )
        
        print(f"Created security group: {security_group_id}")
    except ClientError as e:
        if e.response['Error']['Code'] == 'InvalidGroup.Duplicate':
            # Get existing security group
            sgs = ec2.describe_security_groups(
                Filters=[
                    {'Name': 'group-name', 'Values': ['redshift-workshop-sg']},
                    {'Name': 'vpc-id', 'Values': [vpc_id]}
                ]
            )
            security_group_id = sgs['SecurityGroups'][0]['GroupId']
            print(f"Using existing security group: {security_group_id}")
        else:
            print(f"Error creating security group: {e}")
            raise
    
    # Create Redshift cluster
    try:
        response = redshift.create_cluster(
            ClusterIdentifier=cluster_identifier,
            NodeType=node_type,
            NumberOfNodes=number_of_nodes,
            DBName=db_name,
            MasterUsername=master_username,
            MasterUserPassword=master_password,
            VpcSecurityGroupIds=[security_group_id],
            PubliclyAccessible=True,  # For workshop purposes
            IamRoles=[redshift_role_arn]
        )
        
        print(f"Creating Redshift cluster: {cluster_identifier}")
        return True
        
    except ClientError as e:
        if e.response['Error']['Code'] == 'ClusterAlreadyExists':
            print(f"Redshift cluster {cluster_identifier} already exists")
            return True
        else:
            print(f"Error creating Redshift cluster: {e}")
            raise

# Create the Redshift cluster
create_redshift_cluster()

Using default VPC: vpc-6f769c04
Created security group: sg-0f003daafe19fedfa
Creating Redshift cluster: redshift-workshop


True

### 2.4 Wait for Cluster to be Available

In [6]:
def wait_for_cluster_available():
    """Wait for the Redshift cluster to be in 'available' state"""
    redshift = boto3.client('redshift', region_name=region)
    
    print(f"Waiting for cluster {cluster_identifier} to be available...")
    
    while True:
        response = redshift.describe_clusters(ClusterIdentifier=cluster_identifier)
        status = response['Clusters'][0]['ClusterStatus']
        
        if status == 'available':
            print(f"Cluster {cluster_identifier} is now available")
            endpoint = response['Clusters'][0]['Endpoint']['Address']
            port = response['Clusters'][0]['Endpoint']['Port']
            return endpoint, port
        
        print(f"Cluster status: {status}. Waiting...")
        time.sleep(30)  # Check every 30 seconds for workshop purposes

# Wait for cluster to be available
cluster_endpoint, cluster_port = wait_for_cluster_available()
print(f"Cluster endpoint: {cluster_endpoint}")
print(f"Cluster port: {cluster_port}")

Waiting for cluster redshift-workshop to be available...
Cluster status: creating. Waiting...
Cluster status: creating. Waiting...
Cluster status: creating. Waiting...
Cluster status: creating. Waiting...
Cluster redshift-workshop is now available
Cluster endpoint: redshift-workshop.c1e1ajkaqemc.ap-south-1.redshift.amazonaws.com
Cluster port: 5439


## 3. Database Setup and Data Loading

### 3.1 Create a Function to Execute SQL Commands

In [7]:
def execute_sql(sql, fetch=False):
    """Execute SQL commands on the Redshift cluster"""
    conn = None
    result = None
    
    try:
        # Connect to the Redshift cluster
        conn = psycopg2.connect(
            host=cluster_endpoint,
            port=cluster_port,
            dbname=db_name,
            user=master_username,
            password=master_password
        )
        
        # Set isolation level to autocommit
        conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
        
        # Create a cursor
        cur = conn.cursor()
        
        # Execute the SQL command
        cur.execute(sql)
        
        # Fetch results if requested
        if fetch:
            result = cur.fetchall()
        
        # Close cursor
        cur.close()
        
        return result
        
    except Exception as e:
        print(f"Database error: {e}")
        raise
        
    finally:
        # Close connection
        if conn:
            conn.close()

### 3.2 Create Schema and Tables

In [13]:
 #First, create the schema
create_schema_sql = """
CREATE SCHEMA IF NOT EXISTS analytics;
"""
execute_sql(create_schema_sql)

# Then create the tables
create_tables_sql = """
-- Set search path
SET search_path TO analytics, public;

-- Create customers dimension table with ALL distribution
CREATE TABLE IF NOT EXISTS analytics.customers (
    customer_id INTEGER PRIMARY KEY,
    customer_name VARCHAR(100),
    email VARCHAR(100),
    city VARCHAR(50),
    state VARCHAR(50),
    country VARCHAR(50),
    registration_date DATE
)
DISTSTYLE ALL
SORTKEY(customer_id);

-- Create products dimension table with ALL distribution
CREATE TABLE IF NOT EXISTS analytics.products (
    product_id INTEGER PRIMARY KEY,
    product_name VARCHAR(100) NOT NULL,
    category VARCHAR(50) NOT NULL,
    price DECIMAL(10,2) NOT NULL,
    cost DECIMAL(10,2)
)
DISTSTYLE ALL
SORTKEY(product_id);

-- Create sales fact table with KEY distribution
CREATE TABLE IF NOT EXISTS analytics.sales (
    sale_id INTEGER PRIMARY KEY,
    customer_id INTEGER NOT NULL REFERENCES analytics.customers(customer_id),
    product_id INTEGER NOT NULL REFERENCES analytics.products(product_id),
    sale_date DATE NOT NULL,
    quantity INTEGER NOT NULL,
    unit_price DECIMAL(10,2) NOT NULL,
    total_amount DECIMAL(10,2) NOT NULL
)
DISTKEY(customer_id)
COMPOUND SORTKEY(sale_date, customer_id);
"""

execute_sql(create_tables_sql)
print("Created schema and tables")

Created schema and tables


### 3.3 Load Data from S3 Using COPY

In [14]:
# Load data into tables using COPY command
load_data_sql = f"""
-- Load customers data
COPY analytics.customers
FROM 's3://{s3_bucket}/{sample_data_prefix}customers.csv'
IAM_ROLE '{redshift_role_arn}'
FORMAT AS CSV DELIMITER ',' IGNOREHEADER 1
REGION '{region}';

-- Load products data
COPY analytics.products
FROM 's3://{s3_bucket}/{sample_data_prefix}products.csv'
IAM_ROLE '{redshift_role_arn}'
FORMAT AS CSV DELIMITER ',' IGNOREHEADER 1
REGION '{region}';

-- Load sales data
COPY analytics.sales
FROM 's3://{s3_bucket}/{sample_data_prefix}sales.csv'
IAM_ROLE '{redshift_role_arn}'
FORMAT AS CSV DELIMITER ',' IGNOREHEADER 1
REGION '{region}';
"""

execute_sql(load_data_sql)
print("Loaded data from S3 to Redshift tables")

Loaded data from S3 to Redshift tables


In [2]:
### 3.4 Verify Data Load

In [15]:
# Verify data was loaded correctly
verify_load_sql = """
SELECT 'customers' AS table_name, COUNT(*) AS row_count FROM analytics.customers
UNION ALL
SELECT 'products', COUNT(*) FROM analytics.products
UNION ALL
SELECT 'sales', COUNT(*) FROM analytics.sales;
"""

row_counts = execute_sql(verify_load_sql, fetch=True)
print("Table row counts:")
for row in row_counts:
    print(f"  {row[0]}: {row[1]} rows")

Table row counts:
  products: 50 rows
  sales: 1000 rows
  customers: 100 rows


## 4. Running Analytical Queries

### 4.1 Create a Materialized View for Common Aggregations

In [16]:
# Create a materialized view for sales by category
materialized_view_sql = """
-- Create materialized view for daily sales by category
CREATE MATERIALIZED VIEW analytics.daily_sales_by_category AS
SELECT 
    s.sale_date,
    p.category,
    COUNT(DISTINCT s.customer_id) AS unique_customers,
    SUM(s.quantity) AS units_sold,
    SUM(s.total_amount) AS total_revenue,
    SUM(s.total_amount - (s.quantity * p.cost)) AS gross_profit
FROM analytics.sales s
JOIN analytics.products p ON s.product_id = p.product_id
GROUP BY s.sale_date, p.category;
"""

execute_sql(materialized_view_sql)
print("Created materialized view for sales by category")

Created materialized view for sales by category


### 4.2 Run Analytical Queries

In [17]:
# Run sample analytical queries

# Query 1: Top selling products
query1 = """
SELECT 
    p.product_id,
    p.product_name,
    p.category,
    SUM(s.quantity) AS total_quantity_sold,
    SUM(s.total_amount) AS total_revenue
FROM analytics.sales s
JOIN analytics.products p ON s.product_id = p.product_id
GROUP BY p.product_id, p.product_name, p.category
ORDER BY total_revenue DESC
LIMIT 10;
"""

# Execute query and display results
results1 = execute_sql(query1, fetch=True)
print("Top 10 selling products by revenue:")
for row in results1:
    print(f"  Product ID: {row[0]}, Name: {row[1]}, Category: {row[2]}, Quantity: {row[3]}, Revenue: ${row[4]:.2f}")

# Query 2: Monthly sales trends
query2 = """
SELECT 
    DATE_TRUNC('month', sale_date) AS month,
    SUM(total_amount) AS monthly_revenue,
    COUNT(DISTINCT customer_id) AS unique_customers
FROM analytics.sales
GROUP BY DATE_TRUNC('month', sale_date)
ORDER BY month;
"""

# Execute query and display results
results2 = execute_sql(query2, fetch=True)
print("\nMonthly sales trends:")
for row in results2:
    print(f"  Month: {row[0]}, Revenue: ${row[1]:.2f}, Unique Customers: {row[2]}")

# Query 3: Customer spend by category
query3 = """
SELECT 
    c.city,
    c.state,
    p.category,
    COUNT(DISTINCT s.customer_id) AS customer_count,
    SUM(s.total_amount) AS total_spent
FROM analytics.sales s
JOIN analytics.customers c ON s.customer_id = c.customer_id
JOIN analytics.products p ON s.product_id = p.product_id
GROUP BY c.city, c.state, p.category
ORDER BY total_spent DESC
LIMIT 15;
"""

# Execute query and display results
results3 = execute_sql(query3, fetch=True)
print("\nTop customer segments by spend:")
for row in results3:
    print(f"  Location: {row[0]}, {row[1]}, Category: {row[2]}, Customers: {row[3]}, Total Spent: ${row[4]:.2f}")

Top 10 selling products by revenue:
  Product ID: 49, Name: Product 49, Category: Books, Quantity: 63, Revenue: $30839.13
  Product ID: 50, Name: Product 50, Category: Toys, Quantity: 56, Revenue: $27972.00
  Product ID: 47, Name: Product 47, Category: Clothing, Quantity: 56, Revenue: $26293.68
  Product ID: 33, Name: Product 33, Category: Home, Quantity: 73, Revenue: $24065.91
  Product ID: 40, Name: Product 40, Category: Toys, Quantity: 60, Revenue: $23976.00
  Product ID: 46, Name: Product 46, Category: Electronics, Quantity: 52, Revenue: $23896.08
  Product ID: 48, Name: Product 48, Category: Home, Quantity: 49, Revenue: $23496.48
  Product ID: 42, Name: Product 42, Category: Clothing, Quantity: 55, Revenue: $23076.90
  Product ID: 34, Name: Product 34, Category: Books, Quantity: 67, Revenue: $22757.22
  Product ID: 35, Name: Product 35, Category: Toys, Quantity: 63, Revenue: $22027.91

Monthly sales trends:
  Month: 2023-01-01 00:00:00, Revenue: $66473.42, Unique Customers: 68
  M

### 4.3 Compare Query Performance With and Without Materialized View

In [18]:
# Compare performance between direct query and materialized view
import time

# Direct query (without materialized view)
direct_query = """
SELECT 
    s.sale_date,
    p.category,
    COUNT(DISTINCT s.customer_id) AS unique_customers,
    SUM(s.quantity) AS units_sold,
    SUM(s.total_amount) AS total_revenue,
    SUM(s.total_amount - (s.quantity * p.cost)) AS gross_profit
FROM analytics.sales s
JOIN analytics.products p ON s.product_id = p.product_id
GROUP BY s.sale_date, p.category
ORDER BY s.sale_date, total_revenue DESC
LIMIT 100;
"""

# Materialized view query
mv_query = """
SELECT *
FROM analytics.daily_sales_by_category
ORDER BY sale_date, total_revenue DESC
LIMIT 100;
"""

print("Testing query performance...")

# Test direct query
start_time = time.time()
execute_sql(direct_query, fetch=True)
direct_duration = time.time() - start_time
print(f"Direct query duration: {direct_duration:.2f} seconds")

# Test materialized view query
start_time = time.time()
execute_sql(mv_query, fetch=True)
mv_duration = time.time() - start_time
print(f"Materialized view query duration: {mv_duration:.2f} seconds")

# Calculate improvement
if direct_duration > 0 and mv_duration > 0:
    improvement = ((direct_duration - mv_duration) / direct_duration) * 100
    print(f"Performance improvement with materialized view: {improvement:.2f}%")

Testing query performance...
Direct query duration: 9.96 seconds
Materialized view query duration: 8.19 seconds
Performance improvement with materialized view: 17.72%


## 5. Examine Execution Plans

### 5.1 Analyze Query Execution Plan

In [19]:
# Get execution plan for an analytical query
explain_query = """
EXPLAIN
SELECT 
    c.state,
    p.category,
    DATE_TRUNC('month', s.sale_date) AS month,
    COUNT(DISTINCT s.customer_id) AS unique_customers,
    SUM(s.total_amount) AS total_revenue
FROM analytics.sales s
JOIN analytics.customers c ON s.customer_id = c.customer_id
JOIN analytics.products p ON s.product_id = p.product_id
GROUP BY c.state, p.category, DATE_TRUNC('month', s.sale_date)
ORDER BY c.state, p.category, month;
"""

# Execute and display the execution plan
execution_plan = execute_sql(explain_query, fetch=True)
print("Query Execution Plan:")
for row in execution_plan:
    print(row[0])

Query Execution Plan:
XN Merge  (cost=1000000000113.02..1000000000113.52 rows=200 width=230)
  Merge Key: grvar_1, grvar_2, grvar_3
  ->  XN Network  (cost=1000000000113.02..1000000000113.52 rows=200 width=230)
        Send to leader
        ->  XN Sort  (cost=1000000000113.02..1000000000113.52 rows=200 width=230)
              Sort Key: grvar_1, grvar_2, grvar_3
              ->  XN HashAggregate  (cost=104.38..105.38 rows=200 width=230)
                    ->  XN Subquery Scan subq  (cost=74.38..91.88 rows=1000 width=230)
                          ->  XN HashAggregate  (cost=74.38..81.88 rows=1000 width=40)
                                ->  XN Hash Join DS_DIST_ALL_NONE  (cost=1.88..61.88 rows=1000 width=40)
                                      Hash Cond: ("outer".customer_id = "inner".customer_id)
                                      ->  XN Hash Join DS_DIST_ALL_NONE  (cost=0.62..33.12 rows=1000 width=34)
                                            Hash Cond: ("outer".product_id

### 5.2 Examine Table Design

In [21]:
table_info_query = """
SELECT 
    "schema" as table_schema,
    "table" as table_name,
    diststyle,
    sortkey1,
    encoded,
    size/1024/1024 as size_mb
FROM svv_table_info
WHERE "schema" = 'analytics'
ORDER BY "schema", "table";
"""

# Execute and display table information
table_info = execute_sql(table_info_query, fetch=True)
print("Redshift Table Information:")
for row in table_info:
    print(f"Table: {row[0]}.{row[1]}")
    print(f"  Distribution Style: {row[2]}")
    print(f"  Sort Key: {row[3]}")
    print(f"  Compression Encoded: {row[4]}")
    print(f"  Size: {row[5]:.2f} MB")
    print("")

Redshift Table Information:
Table: analytics.customers
  Distribution Style: ALL
  Sort Key: customer_id
  Compression Encoded: Y, AUTO(ENCODE)
  Size: 0.00 MB

Table: analytics.daily_sales_by_category
  Distribution Style: EVEN
  Sort Key: None
  Compression Encoded: Y
  Size: 0.00 MB

Table: analytics.products
  Distribution Style: ALL
  Sort Key: product_id
  Compression Encoded: Y, AUTO(ENCODE)
  Size: 0.00 MB

Table: analytics.sales
  Distribution Style: KEY(customer_id)
  Sort Key: sale_date
  Compression Encoded: Y, AUTO(ENCODE)
  Size: 0.00 MB




## 6. Clean Up Resources

### 6.1 Delete Redshift Cluster

In [22]:
def delete_redshift_cluster():
    """Delete the Redshift cluster"""
    redshift = boto3.client('redshift', region_name=region)
    
    try:
        # Delete the cluster without final snapshot
        redshift.delete_cluster(
            ClusterIdentifier=cluster_identifier,
            SkipFinalClusterSnapshot=True
        )
        
        print(f"Deleting Redshift cluster: {cluster_identifier}")
        return True
        
    except ClientError as e:
        print(f"Error deleting Redshift cluster: {e}")
        return False

# Delete the Redshift cluster
delete_redshift_cluster()

Deleting Redshift cluster: redshift-workshop


True

### 6.2 Wait for Cluster Deletion

In [ ]:
def wait_for_cluster_deletion():
    """Wait for the Redshift cluster to be deleted"""
    redshift = boto3.client('redshift', region_name=region)
    
    print(f"Waiting for cluster {cluster_identifier} to be deleted...")
    
    while True:
        try:
            response = redshift.describe_clusters(ClusterIdentifier=cluster_identifier)
            status = response['Clusters'][0]['ClusterStatus']
            
            print(f"Cluster status: {status}. Waiting...")
            time.sleep(30)  # Check every 30 seconds
            
        except ClientError as e:
            if e.response['Error']['Code'] == 'ClusterNotFound':
                print(f"Cluster {cluster_identifier} has been deleted")
                return True
            else:
                print(f"Error checking cluster status: {e}")
                return False

# Wait for cluster deletion
wait_for_cluster_deletion()

Waiting for cluster redshift-workshop to be deleted...
Cluster status: deleting. Waiting...
Cluster status: deleting. Waiting...


### 6.3 Delete S3 Bucket and IAM Role

In [ ]:
def clean_up_resources():
    """Delete S3 bucket and IAM role"""
    # Delete S3 bucket contents and bucket
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(s3_bucket)
    
    try:
        print(f"Deleting all objects from bucket {s3_bucket}")
        bucket.objects.all().delete()
        
        print(f"Deleting bucket {s3_bucket}")
        bucket.delete()
    except ClientError as e:
        print(f"Error deleting S3 bucket: {e}")
    
    # Detach policy and delete IAM role
    iam = boto3.client('iam')
    
    try:
        print("Detaching policy from IAM role")
        iam.detach_role_policy(
            RoleName='RedshiftWorkshopRole',
            PolicyArn='arn:aws:iam::aws:policy/AmazonS3ReadOnlyAccess'
        )
        
        print("Deleting IAM role")
        iam.delete_role(RoleName='RedshiftWorkshopRole')
    except ClientError as e:
        print(f"Error cleaning up IAM role: {e}")
    
    print("Resource cleanup completed")

# Clean up resources
clean_up_resources()


## Workshop Summary

In this hands-on workshop, you've successfully:

1. Created an IAM role for Redshift to access S3
2. Set up an S3 bucket with sample e-commerce data
3. Deployed an Amazon Redshift cluster
4. Created optimized tables with appropriate distribution and sort keys
5. Loaded data using the COPY command
6. Created a materialized view to improve query performance
7. Run analytical queries and observed performance differences
8. Examined query execution plans and table design
9. Cleaned up all resources

This workshop demonstrated the core capabilities of Amazon Redshift for data warehousing, from deployment to optimization and querying.